<h1>Import Library

In [14]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler


<h1> Input Data

In [12]:
file_path = "kc_house_data.csv"
data = pd.read_csv(file_path)
data = data.drop(columns=['id', 'date'])

# Pisahkan fitur dan target
X = data.drop(columns=['price']).values
y = data['price'].values

# Misalnya dataset memiliki fitur X dan label y
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)  # 20% untuk val + test
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)  # Bagi 10% - 10%

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_train = y_train.astype("float32")
y_test = y_test.astype("float32")

# Cek ukuran data
print(f"Train size: {len(X_train)} ({len(X_train)/len(X)*100:.1f}%)")
print(f"Validation size: {len(X_val)} ({len(X_val)/len(X)*100:.1f}%)")
print(f"Test size: {len(X_test)} ({len(X_test)/len(X)*100:.1f}%)")

print(data.head(10))


Train size: 17290 (80.0%)
Validation size: 2161 (10.0%)
Test size: 2162 (10.0%)
       price  bedrooms  bathrooms  sqft_living  sqft_lot  floors  waterfront  \
0   221900.0         3       1.00         1180      5650     1.0           0   
1   538000.0         3       2.25         2570      7242     2.0           0   
2   180000.0         2       1.00          770     10000     1.0           0   
3   604000.0         4       3.00         1960      5000     1.0           0   
4   510000.0         3       2.00         1680      8080     1.0           0   
5  1225000.0         4       4.50         5420    101930     1.0           0   
6   257500.0         3       2.25         1715      6819     2.0           0   
7   291850.0         3       1.50         1060      9711     1.0           0   
8   229500.0         3       1.00         1780      7470     1.0           0   
9   323000.0         3       2.50         1890      6560     2.0           0   

   view  condition  grade  sqft_above  

<h1>XGBoost

In [15]:
import xgboost as xgb

# Konversi ke DMatrix (format khusus XGBoost)
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)

# Parameter XGBoost (sederhana)
params = {
    "objective": "reg:squarederror",  # Regression task
    "eval_metric": "mae",  # Mean Absolute Error
    "learning_rate": 0.1,
    "max_depth": 6,
    "n_jobs": -1  # Gunakan semua core CPU
}

# Training model (3 epoch)
watchlist = [(dtrain, "train"), (dval, "eval")]
model = xgb.train(params, dtrain, num_boost_round=3, evals=watchlist)

# Evaluasi di test set
dtest = xgb.DMatrix(X_test)
y_pred = model.predict(dtest)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error (Test): {mae:.2f}")

[0]	train-mae:213023.25717	eval-mae:321767.16511


[1]	train-mae:196535.93964	eval-mae:552966.19615
[2]	train-mae:181855.67833	eval-mae:745754.93001
Mean Absolute Error (Test): 197079.73


<h1>LightGBM

In [9]:
import lightgbm as lgb
print(lgb.__version__)


4.6.0


<h1>CatBoost

In [10]:
import catboost
print(catboost.__version__)


1.2.7
